# Debugging autoreload

In [1]:
%load_ext autoreload
%autoreload 2

# Load packages

In [1]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
import requests

def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


# Load data

In [2]:
path = f"D:/YandexDisk/Work/bbd/mriya"

yadisk_file_url = "https://disk.yandex.ru/i/CEaw3cqI2Y7J0A"
response = requests.get(
    "https://cloud-api.yandex.net/v1/disk/public/resources/download", 
    params={'public_key': yadisk_file_url}
)
res = response.json()
download_url = res['href']
response = requests.get(download_url)
yadisk_subject_file_name = Path(f"{path}/Испытуемые Яндекс.xlsx")
with open(yadisk_subject_file_name, 'wb') as f:
    f.write(response.content)

In [ ]:
df_params = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Parameters', index_col=0)
df_blood = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Blood', index_col=0)
df_heart = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Heart', index_col=0)
df_blood.insert(3, 'Age', (df_blood['sample_date'] - df_blood['birthday']) / np.timedelta64(1, 'D') / 365.25)
df_heart.insert(3, 'Age', (df_heart['sample_date'] - df_heart['birthday']) / np.timedelta64(1, 'D') / 365.25)

suffixes=('', '_heart')
df = pd.merge(df_blood, df_heart, left_index=True, right_index=True, how='outer', suffixes=suffixes)
cols_cmn = df_blood.columns.intersection(df_heart.columns).to_list()
with pd.ExcelWriter(f"{path}/conflicts.xlsx", engine='xlsxwriter') as writer:
    for col in cols_cmn:
        non_eq_ids = df.index[df[f'{col}{suffixes[0]}'] != df[f'{col}{suffixes[1]}']].to_list()
        df_col = df.loc[non_eq_ids, [f'{col}{suffixes[0]}', f'{col}{suffixes[1]}']]
        df_col.rename(columns={f'{col}{suffixes[0]}': 'Blood', f'{col}{suffixes[1]}': 'Heart'}, inplace=True)
        df_col.to_excel(writer, sheet_name=col)
        
cols_types = df_params['analysis_type'].dropna().unique()
cols_sets = {x: df.columns.intersection(df_params.index[df_params['analysis_type'] == x]).to_list() for x in cols_types}
df = df.loc[:, cols_cmn + list(chain.from_iterable(cols_sets.values()))]

# NaNs analysis

In [4]:
with pd.ExcelWriter(f"{path}/nans.xlsx", engine='xlsxwriter') as writer:
    for col_set, cols in cols_sets.items():
        data = df.loc[:, cols]
        nan_feats = data.isna().sum(axis=0).to_frame(name="Number of NaNs")
        nan_feats["% of NaNs"] = nan_feats["Number of NaNs"] / data.shape[0] * 100
        nan_feats["Number of not-NaNs"] = data.notna().sum(axis=0)
        nan_feats.sort_values(["% of NaNs"], ascending=[True], inplace=True)
        nan_feats.to_excel(writer, sheet_name=col_set)

# Correlation with Age

In [ ]:
hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(6, 3.5))
histplot = sns.histplot(
    data=df[['Age']].dropna(),
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Age",
    color='crimson',
    ax=ax
)
histplot.set(xlim=(0, 120))
plt.savefig(f"{path}/age_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/age_hist.pdf", bbox_inches='tight')
plt.close(fig)

feats_cnt = df_params.index[df_params['data_type'].isin(['decimal', 'integer'])].intersection(list(chain.from_iterable(cols_sets.values()))).to_list()
df_corr = pd.DataFrame(index=feats_cnt, columns=['count', 'rho', 'pval', ])
for f in tqdm(feats_cnt):
    df_tmp = df.loc[:, ['Age', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        df_corr.at[f, 'count'] = df_tmp.shape[0]
        vals_1 = df_tmp.loc[:, 'Age'].values
        vals_2 = df_tmp.loc[:, f].values
        df_corr.at[f, 'rho'], df_corr.at[f, 'pval'] = stats.pearsonr(vals_1, vals_2)
df_corr.dropna(axis=0, how='any', inplace=True)
_, df_corr['pval_fdr_bh'], _, _ = multipletests(df_corr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
df_corr[['feature_name_ru', 'analysis_type', 'description']] = df_params.loc[df_corr.index, ['feature_name_ru', 'analysis_type', 'description']]
df_corr.to_excel(f"{path}/age_pearson.xlsx", index_label="Features")

# Generate datasets

In [ ]:
feats_sets = {
    'Echocardiography': ['Echocardiography'],
    'Sphygmocardiography': ['Sphygmocardiography'],
    'BP': ['BP'],
    'ECG': ['ECG'],
    'Anthropometry': ['Anthropometry'],
    'CBC': ['CBC'],
}

for fs, fls in feats_sets.items():
    Path(f"{path}/{fs}").mkdir(parents=True, exist_ok=True)
    feats = ['sex']
    for fl in fls:
        feats += pd.read_excel(f"{path}/feats_cand.xlsx", sheet_name=fl, index_col=0).index.to_list()
    dataset_feats = df_params.loc[feats, :]
    dataset_feats.to_excel(f"{path}/{fs}/feats.xlsx", index_label="Features")
    dataset_df = df[['Age'] + feats]
    dataset_df.dropna(axis=0, how='any', inplace=True)
    dataset_df.to_excel(f"{path}/{fs}/data.xlsx", index_label="IDs")
    
    feats_cnt = ['Age'] + df_params.index[df_params['data_type'].isin(['decimal', 'integer'])].intersection(feats).to_list()
    df_corr = pd.DataFrame(data=np.zeros(shape=(len(feats_cnt), len(feats_cnt))), index=feats_cnt, columns=feats_cnt)
    for f_id_1 in range(len(feats_cnt)):
        for f_id_2 in range(f_id_1, len(feats_cnt)):
            f_1 = feats_cnt[f_id_1]
            f_2 = feats_cnt[f_id_2]
            if f_id_1 != f_id_2:
                vals_1 = dataset_df.loc[:, f_1].values
                vals_2 = dataset_df.loc[:, f_2].values
                corr, pval = stats.pearsonr(vals_1, vals_2)
                df_corr.at[f_2, f_1] = pval
                df_corr.at[f_1, f_2] = corr
            else:
                df_corr.at[f_2, f_1] = np.nan
    selection = np.tri(df_corr.shape[0], df_corr.shape[1], -1, dtype=bool)
    df_fdr = df_corr.where(selection).stack().reset_index()
    df_fdr.columns = ['row', 'col', 'pval']
    _, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
    nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
    df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
    df_corr_fdr = df_corr.copy()
    for line_id in range(df_fdr.shape[0]):
        df_corr_fdr.loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = -np.log10(df_fdr.at[line_id, 'pval_fdr_bh'])
    df_corr_fdr.to_excel(f"{path}/{fs}/feats_pearsonr.xlsx")
        
    sns.set_theme(style='ticks')
    fig, ax = plt.subplots(figsize=(4.5 + 0.25 * len(feats_cnt), 2.5 + 0.25 * len(feats_cnt)))
    cmap_triu = plt.get_cmap("seismic").copy()
    mask_triu=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool)
    heatmap_diff = sns.heatmap(
        df_corr_fdr,
        mask=mask_triu,
        annot=True,
        fmt=".2f",
        center=0.0,
        cmap=cmap_triu,
        linewidth=0.1,
        linecolor='black',
        annot_kws={"size": 5},
        ax=ax
    )
    ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$", size=13)
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
        
    cmap_tril = plt.get_cmap("viridis").copy()
    cmap_tril.set_under('black')
    mask_tril=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool).T
    heatmap_pval = sns.heatmap(
        df_corr_fdr,
        mask=mask_tril,
        annot=True,
        fmt=".1f",
        vmin=-np.log10(0.05),
        cmap=cmap_tril,
        linewidth=0.1,
        linecolor='black',
        annot_kws={"size": 5},
        ax=ax
    )
    ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$", size=13)
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
    ax.set_xlabel('', fontsize=16)
    ax.set_ylabel('', fontsize=16)
    ax.set_title('', fontsize=16)
    # ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.5, foreground="black")])
    # for tick_label in ax.get_xticklabels():
        # tick_label.set_color(colors_tissues[tick_label.get_text()])
        # ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.5, foreground="black")])
    # for tick_label in ax.get_yticklabels():
        # tick_label.set_color(colors_tissues[tick_label.get_text()])
        # ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.5, foreground="black")])
    plt.savefig(f"{path}/{fs}/feats_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/{fs}/feats_pearsonr.pdf", bbox_inches='tight')
    plt.close(fig)